CME 538
TIME TO IMPORT ALL THE DATA FROM THE DIFFERENT DATA SOURCES! 

In [5]:
import requests
import json
import xml.etree.ElementTree as ET
from API_keys import Translink_API_Key, Metrolinx_API_Key, Transitland_API_Key, STM_API_Key


VANCOUVER

In [6]:
# VANCOUVER - TRANSLINK MANAGED API! 
# https://www.translink.ca/about-us/doing-business-with-translink/app-developer-resources/rtti

def pull_translink_data():
    translink_url = "https://api.translink.ca/rttiapi/v1/buses?apikey="
    
    response = requests.get(translink_url+Translink_API_Key)
    
    translink_Data = response.text
    print(translink_Data)
    
    root = ET.fromstring(translink_Data)

    # Replace 'output.xml' with the desired filename
    output_file = 'translink_data.xml'
    with open(output_file, 'wb') as xml_file:
        xml_file.write(ET.tostring(root, encoding='utf-8'))

    


In [7]:
pull_translink_data()

<Buses xmlns:i="http://www.w3.org/2001/XMLSchema-instance"><Bus><VehicleNo>12001</VehicleNo><TripId>13539101</TripId><RouteNo>049</RouteNo><Direction>WEST</Direction><Destination>UBC</Destination><Pattern>WB1</Pattern><Latitude>49.222067</Latitude><Longitude>-123.024783</Longitude><RecordedTime>10:38:27 am</RecordedTime><RouteMap><Href>https://nb.translink.ca/geodata/049.kmz</Href></RouteMap></Bus><Bus><VehicleNo>12003</VehicleNo><TripId>13590009</TripId><RouteNo>049</RouteNo><Direction>EAST</Direction><Destination>METROTOWN STN</Destination><Pattern>EB1</Pattern><Latitude>49.247100</Latitude><Longitude>-123.232083</Longitude><RecordedTime>10:38:48 am</RecordedTime><RouteMap><Href>https://nb.translink.ca/geodata/049.kmz</Href></RouteMap></Bus><Bus><VehicleNo>12007</VehicleNo><TripId>13590007</TripId><RouteNo>049</RouteNo><Direction>EAST</Direction><Destination>METROTOWN STN</Destination><Pattern>EB1</Pattern><Latitude>49.226467</Latitude><Longitude>-123.005067</Longitude><RecordedTime>

MONTREAL

In [8]:
# MONTREAL - STM MANAGED API! 

def pull_stm_data():
    stm_url = "https://api.stm.info/pub/od/gtfs-rt/ic/v2"
    



OTTAWA

In [9]:
# OTTAWA - OC TRANSPO INFO THROUGH TRANSITLAND API! 

def pull_transitland_octranspo_data():
    
    
    oc_transpo_url = "http://www.octranspo1.com/files/google_transit.zip"


TORONTO

In [15]:
# Toronto Open Data is stored in a CKAN instance. It's APIs are documented here:
# https://docs.ckan.org/en/latest/api/

def pull_ttc_data():
    
    return

def pull_gotrain_data():
    gotrain_url = "http://api.openmetrolinx.com/OpenDataAPI/api/V1/Gtfs/Feed/TripUpdates?key="
    
    response = requests.get(gotrain_url+Metrolinx_API_Key)
    
    gotrain_Data = response.text
    print(gotrain_Data)
    
    # Specify the file path where you want to save the JSON data
    file_path = "gotrain_output.json"  # Replace with your desired file path and filename
    
    # Write the JSON data to the local file
    with open(file_path, "w") as file:
        json.dump(gotrain_Data, file, indent=4)
    
    return

def test_TTC_Open_Data_Call():
    # To hit our API, you'll be making requests to:
    base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca"
    
    # Datasets are called "packages". Each package can contain many "resources"
    # To retrieve the metadata for this package and its resources, use the package name in this page's URL:
    url = base_url + "/api/3/action/package_show"
    params = { "id": "ttc-real-time-next-vehicle-arrival-nvas"}
    package = requests.get(url, params = params).json()
    
    # To get resource data:
    for idx, resource in enumerate(package["result"]["resources"]):
     
           # To get metadata for non datastore_active resources:
           if not resource["datastore_active"]:
               url = base_url + "/api/3/action/resource_show?id=" + resource["id"]
               resource_metadata = requests.get(url).json()
               print(resource_metadata)
               # From here, you can use the "url" attribute to download this file

In [16]:
# call from the APIs
pull_gotrain_data()


{
  "header": {
    "gtfs_realtime_version": "1.0",
    "incrementality": "FULL_DATASET",
    "timestamp": 1699298448
  },
  "entity": [
    {
      "id": "20231106-LW-1019",
      "is_deleted": false,
      "trip_update": {
        "trip": {
          "trip_id": "20231106-LW-1019",
          "route_id": "09231223-LW",
          "direction_id": 1,
          "start_time": "13:17:00",
          "start_date": "20231106",
          "schedule_relationship": "SCHEDULED"
        },
        "vehicle": {
          "id": "667",
          "label": "LW - Aldershot GO",
          "license_plate": ""
        },
        "stop_time_update": [
          {
            "stop_id": "AL",
            "arrival": null,
            "departure": {
              "delay": 18,
              "time": 1699298778,
              "uncertainty": 0
            },
            "schedule_relationship": "SCHEDULED"
          }
        ],
        "timestamp": 1699298430,
        "delay": 18
      },
      "vehicle": null,
    